Here is a big class without Single Responsibility Principle (Seperation of Concerns)

In [ ]:
import re
import smtplib
import logging

class UserManager:
    def __init__(self):
        self.users = []
        self.logger = logging.getLogger('UserManager')
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        self.logger.addHandler(handler)

    def create_user(self, username, email, password):
        if not self.validate_username(username):
            self.logger.error("Invalid username")
            return "Invalid username"

        if not self.validate_email(email):
            self.logger.error("Invalid email")
            return "Invalid email"

        if not self.validate_password(password):
            self.logger.error("Invalid password")
            return "Invalid password"

        user = {"username": username, "email": email, "password": password}
        self.users.append(user)
        self.logger.info(f"User created: {username}")
        self.send_welcome_email(email)
        self.save_to_database(user)
        return "User created successfully"

    def validate_username(self, username):
        return len(username) >= 3

    def validate_email(self, email):
        return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None

    def validate_password(self, password):
        return len(password) >= 8

    def send_welcome_email(self, email):
        sender = "admin@example.com"
        message = f"Subject: Welcome\n\nWelcome to our platform!"
        try:
            with smtplib.SMTP('localhost') as server:
                server.sendmail(sender, email, message)
            self.logger.info(f"Welcome email sent to {email}")
        except Exception as e:
            self.logger.error(f"Failed to send welcome email to {email}: {e}")

    def save_to_database(self, user):
        # Simulate saving to a database
        self.logger.info(f"User {user['username']} saved to database")

    def log_activity(self, activity):
        self.logger.info(activity)


Now Lets seperatet this class

In [ ]:
# imports
import re
import smtplib
import logging

In [ ]:
class LoggerManager:
  def __init__(self):
        self.logger = logging.getLogger('UserManager')
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        self.logger.addHandler(handler)
  def log_activity_info(self, activity):
        self.logger.info(activity)
  def log_activity_error(self, activity):
        self.logger.error(activity)

In [ ]:
class EmailManager:
  def __init__(self):
    self.sender = "admin@example.com"
    self.logger=LoggerManager()
  def validate_email(self, email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None
  def send_welcome_email(self, email):
      sender = "admin@example.com"
      message = f"Subject: Welcome\n\nWelcome to our platform!"
      try:
          with smtplib.SMTP('localhost') as server:
              server.sendmail(sender, email, message)
          self.logger.log_activity_info(f"Welcome email sent to {email}")
      except Exception as e:
          self.logger.log_activity_error(f"Failed to send welcome email to {email}: {e}")

In [ ]:
class DB:
  def __init__(self):
    self.db_dictionary={}
    self.logger=LoggerManager()
  def save_to_database(self, user):
    print(user)
    self.db_dictionary[user['username']]=user
    self.logger.log_activity_info(f"User {user['username']} saved to database")

In [ ]:
class UserManager:
  def __ini__(self):
    self.users = []
    self.logger=LoggerManager()
    self.email_manager=EmailManager()
    self.db=DB()

  def validate_username(self, username):
      return len(username) >= 3
  def validate_password(self, password):
      return len(password) >= 8

  def create_user(self, username, email, password):
    if not self.validate_username(username):
        self.logger.log_activity_error("Invalid username")
        return "Invalid username"

    if not self.email_manager.validate_email(email):
        self.logger.log_activity_error("Invalid email")
        return "Invalid email"

    if not self.validate_password(password):
        self.logger.log_activity_error("Invalid password")
        return "Invalid password"

    user = {"username": username, "email": email, "password": password}
    self.users.append(user)
    self.logger.log_activity_info(f"User created: {username}")
    self.email_manager.send_welcome_email(email)
    self.db.save_to_database(user)
    return "User created successfully"

Here Let's  have a class that break open-close principle (OCP)

In [ ]:
class Employee:
  def __init__(self,name:str,salary:str):
    self.name=name
    self.salary=salary

class Tester(Employee):
  def __init__(self,name:str,salary:str):
    super().__init__(name,salary)
  def test(self):
    print(f"{self.name} is testing")

class Developer(Employee):
  def __init__(self,name:str,salary:str):
    super().__init__(name,salary)
  def develpe(self):
    print(f"{self.name} is developing")

class Company:
  def __init__(self,name):
    self.name=name
  def work(self,employee):
    if isinstance(employee,Tester):
      employee.test()
    elif isinstance(employee,Developer):
      employee.develpe()
    else:
      raise Exception("Unknown employee")


In [ ]:
tester1=Tester(name="tester1",salary="1")
tester2=Tester(name="tester2",salary="2")
developer1=Developer(name="developer1",salary="3")
developer2=Developer(name="developer2",salary="4")
company=Company(name="company1")
company.work(tester1)
company.work(tester2)
company.work(developer1)
company.work(developer2)

tester1 is testing
tester2 is testing
developer1 is developing
developer2 is developing


as you can see , we have two type of employee . developer and tester .
now if we want to add another type of employee we should change the code in company which means we have to do "modification" and this will break the "OCP" so what we do ? use abstract classes

In [ ]:
from abc import ABC , abstractmethod
class EmployeeABC(ABC):
  # abstract classes can not be instancitated
  def __init__(self,name:str,salary:str):
    print("in the employee init")
    self.name=name
    self.salary=salary

  @abstractmethod
  def work(self):
    pass

class TesterABC(EmployeeABC):
  def __init__(self,name:str,salary:str):
    super().__init__(name,salary)
  def test(self):
    print(f"{self.name} is testing")
  def work(self):
    self.test()

class DeveloperABC(EmployeeABC):
  def __init__(self,name:str,salary:str):
    super().__init__(name,salary)
  def develpe(self):
    print(f"{self.name} is developing")
  def work(self):
    self.develpe()

class TeamLeaderABC(EmployeeABC):
  def __init__(self,name:str,salary:str):
    super().__init__(name,salary)
  def leader(self):
    print(f"{self.name} is leader")
  def work(self):
    self.leader()


class CompanyABC:
  def __init__(self,name):
    self.name=name
  def work(self,employee:Employee):
    employee.work()


In [ ]:
tester1=TesterABC(name="tester1",salary="1")
tester2=TesterABC(name="tester2",salary="2")
developer1=DeveloperABC(name="developer1",salary="3")
developer2=DeveloperABC(name="developer2",salary="4")
leader1=TeamLeaderABC(name="leader1",salary="5")
leader2=TeamLeaderABC(name="leader2",salary="6")
company=CompanyABC(name="company1")
company.work(tester1)
company.work(tester2)
company.work(developer1)
company.work(developer2)
company.work(leader1)
company.work(leader2)

in the employee init
in the employee init
in the employee init
in the employee init
in the employee init
in the employee init
tester1 is testing
tester2 is testing
developer1 is developing
developer2 is developing
leader1 is leader
leader2 is leader


what about have another example? rewrite this class with OCP

In [ ]:
class PaymentProcessor:
    def process_payment(self, payment_type, amount):
        if payment_type == "credit_card":
            self.process_credit_card_payment(amount)
        elif payment_type == "paypal":
            self.process_paypal_payment(amount)
        elif payment_type == "apple_pay":
            self.process_apple_pay_payment(amount)
        else:
            raise ValueError("Unsupported payment type")

    def process_credit_card_payment(self, amount):
        print(f"Processing credit card payment of {amount}")

    def process_paypal_payment(self, amount):
        print(f"Processing PayPal payment of {amount}")

    def process_apple_pay_payment(self, amount):
        print(f"Processing Apple Pay payment of {amount}")


here is the above code but written with ocp

In [ ]:
from abc import ABC , abstractmethod
class PaymentABC(ABC):
  def __init__(self,amount:str):
    self.amount=amount
  @abstractmethod
  def process_payment(self):
    pass

class CreditCardPayment(PaymentABC):
  def __init__(self,amount:str):
    super().__init__(amount)
  def process_credit_card_payment(self):
     print(f"Processing credit card payment of {self.amount}")
  def process_payment(self):
    self.process_credit_card_payment()

class PayPalPayment(PaymentABC):
  def __init__(self,amount:str):
    super().__init__(amount)
  def process_paypal_payment(self):
      print(f"Processing PayPal payment of {self.amount}")
  def process_payment(self):
    self.process_paypal_payment()


class ApplePayPayment(PaymentABC):
  def __init__(self,amount:str):
    super().__init__(amount)
  def process_apple_pay_payment(self):
      print(f"Processing Apple Pay payment of {self.amount}")
  def process_payment(self):
    self.process_apple_pay_payment()


class PaymentProcessorABC:
  def process_payment(self,payment_type):
    payment_type.process_payment()


In [ ]:
# prompt: create objects from classes of the above cell

credit_card_payment = CreditCardPayment("100")
paypal_payment = PayPalPayment("50")
apple_pay_payment = ApplePayPayment("75")

payment_processor = PaymentProcessorABC()
payment_processor.process_payment(credit_card_payment)
payment_processor.process_payment(paypal_payment)
payment_processor.process_payment(apple_pay_payment)


Processing credit card payment of 100
Processing PayPal payment of 50
Processing Apple Pay payment of 75


ok now lets have a code so we can rewrite it with Liskov substitution principle ( lsp) which said that the objects of child class can be replcaed with the object of parent class


In [ ]:
class Bird:
    def fly(self):
        return "I am flying"

class Penguin(Bird):
    def fly(self):
        raise NotImplementedError("Penguins can't fly")

def let_bird_fly(bird: Bird):
    try:
        return bird.fly()
    except NotImplementedError as e:
        return str(e)

# Creating instances
sparrow = Bird()
penguin = Penguin()

# Using the let_bird_fly function
print(let_bird_fly(sparrow))  # Output: I am flying
print(let_bird_fly(penguin))  # Output: Penguins can't fly
# this code breaks lsp

I am flying
Penguins can't fly


now here with lsp

In [ ]:
# solution : that method which have different implementations should be another abstract class and abstract method and then inherit
# here is fly
from abc import ABC , abstractmethod
class Bird(ABC):
  @abstractmethod
  def fly(self):
    pass

class CanFly(Bird):
  def fly(self):
    return "I am flying"
class CanNotFly(Bird):
  def fly(self):
    return "I can not fly"

class Penguin(CanNotFly):
  pass
class Sparrow(CanFly):
  pass


def let_bird_fly(bird: Bird):
    return bird.fly()

sparrow = Sparrow()
penguin = Penguin()

# Using the let_bird_fly function
print(let_bird_fly(sparrow))  # Output: I am flying
print(let_bird_fly(penguin))

I am flying
I can not fly


lets have another example

In [ ]:
# this code violates the lsp
from abc import ABC, abstractmethod

class Report(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        pass

class SalesReport(Report):
    def generate_report(self) -> str:
        return "Sales Report Data"

class ErrorReport(Report):
    def generate_report(self) -> str:
        raise Exception("Error Report cannot be generated")

def print_report(report: Report):
    try:
        print(report.generate_report())
    except Exception as e:
        print(f"Report error: {e}")

# Usage
sales_report = SalesReport()
error_report = ErrorReport()

print_report(sales_report)  # Output: Sales Report Data
print_report(error_report)  # Output: Report error: Error Report cannot be generated


Sales Report Data
Report error: Error Report cannot be generated


here is the correct code

In [ ]:
from abc import ABC, abstractmethod

class Report(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        pass
class CanGenerateReport(Report):
    def generate_report(self) -> str:
        return f"{self.__class__.__name__} Repot data"
class CanNotGenerateReport(Report):
    def generate_report(self) -> str:
        return f"{self.__class__.__name__} Report cannot be generated"

class SalesReport(CanGenerateReport):
    pass
class ErrorReport(CanNotGenerateReport):
    pass


def print_report(report: Report):
        print(report.generate_report())

sales_report = SalesReport()
error_report = ErrorReport()

print_report(sales_report)  # Output: Sales Report Data
print_report(error_report)

SalesReport Repot data
ErrorReport Report cannot be generated


lets have the third examlpe

In [ ]:
from abc import ABC, abstractmethod


class Member(ABC):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @abstractmethod
    def save_database(self):
        pass

    @abstractmethod
    def pay(self):
        pass


class Teacher(Member):
    def __init__(self, name, age, teacher_id):
        super().__init__(name, age)
        self.teacher_id = teacher_id

    def save_database(self):
        print("Saving teacher data to database")

    def pay(self):
        print("Paying")


class Manager(Member):
    def __init__(self, name, age, manager_id):
        super().__init__(name, age)
        self.manager_id = manager_id

    def save_database(self):
        print("Saving manager data to database")

    def pay(self):
        print("Paying")


class Student(Member):
    def __init__(self, name, age, student_id):
        super().__init__(name, age)
        self.student_id = student_id

    def save_database(self):
        print("Saving student data to database")

    def pay(self):
        raise NotImplementedError("It is free for students!")

in this example method named :pay , has different type of implementation . lets fix it by create is an abstract method . but it is alreadyan abstract method so lets make it a seperate one not the one in the class and the do not student inherit it


In [ ]:
from abc import ABC, abstractmethod

class Payment(ABC):
  @abstractmethod
  def pay(self):
    pass
class Member(ABC):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @abstractmethod
    def save_database(self):
        pass



class Teacher(Member,Payment):
    def __init__(self, name, age, teacher_id):
        super().__init__(name, age)
        self.teacher_id = teacher_id

    def save_database(self):
        print("Saving teacher data to database")

    def pay(self):
        print("Paying")


class Manager(Member,Payment):
    def __init__(self, name, age, manager_id):
        super().__init__(name, age)
        self.manager_id = manager_id

    def save_database(self):
        print("Saving manager data to database")

    def pay(self):
        print("Paying")


class Student(Member):
    def __init__(self, name, age, student_id):
        super().__init__(name, age)
        self.student_id = student_id

    def save_database(self):
        print("Saving student data to database")


the next principle is Interface Segregation , which said that do not force user to use all the methods and attributes writtine in an interface (somehow indirectly it is taking about to have single responsiblity but it is not the same as srp )

In [ ]:
class Worker:
    def work(self):
        pass

    def eat(self):
        pass

    def sleep(self):
        pass

class Robot(Worker):
    def work(self):
        print("Robot is working")

    def eat(self):
        # Robots don't eat, but this method must be implemented
        pass

    def sleep(self):
        # Robots don't sleep, but this method must be implemented
        pass

class Human(Worker):
    def work(self):
        print("Human is working")

    def eat(self):
        print("Human is eating")

    def sleep(self):
        print("Human is sleeping")


the above code obligate user to see (have) methods that are not usefull for it . so lets delete them in a beautiful way with abstraction

In [ ]:
from abc import ABC, abstractmethod
class Work(ABC):
  @abstractmethod
  def work(self):
    pass

class Eat(ABC):
  @abstractmethod
  def eat(self):
    pass

class Sleep(ABC):
  @abstractmethod
  def sleep(self):
    pass

class Worker:
  # this class can be deleted also
  pass

class Robot(Work):
  def work(self):
    print("Robot is working")

class Human(Work,Eat,Sleep):
  def work(self):
    print("Human is working")
  def eat(self):
    print("Human is eating")
  def sleep(self):
    print("Human is sleeping")

the last principle is Dependency Inversion Principle (DIP)
These principle said that we have two kind f modules , high levels which are responsible for business logic and core and functionalities (the high level modules just define that we have these inputs and outputs . do not define the implementation) then we have low level modules which define the implementation of those functionalities that high levels wants . now how to connect these two together ? with abstraction

high levels told the abstractions that there are these functionalites whith these inputs and output . abstraction told the low levels to implement them. so we have an abstraction with the name of functionalities and their method signature , then those low levels inerits from abstraction and implement the functionalities , then the high levels depend on the abstraction class by using its class as and obj and use the functionalities

In [ ]:
# this code breaks the DIP
class EmailService:
    def send_email(self, message):
        print(f"Sending email: {message}")

class SMSService:
    def send_sms(self, message):
        print(f"Sending SMS: {message}")

class NotificationService:
    def __init__(self, use_email=True):
        if use_email:
            self.service = EmailService()
        else:
            self.service = SMSService()

    def send_notification(self, message):
        if isinstance(self.service, EmailService):
            self.service.send_email(message)
        elif isinstance(self.service, SMSService):
            self.service.send_sms(message)

# Example usage
notification_service = NotificationService(use_email=True)
notification_service.send_notification("Hello via Email!")

notification_service = NotificationService(use_email=False)
notification_service.send_notification("Hello via SMS!")


Sending email: Hello via Email!
Sending SMS: Hello via SMS!


the code above breaks the DIP , lets rewrite it


In [ ]:
from abc import ABC , abstractmethod
class Messanger(ABC):
  @abstractmethod
  def send_message(self,message):
    pass

class Email(Messanger):
  def send_message(self,message):
        print(f"Sending email: {message}")
class SMS(Messanger):
  def send_message(self,message):
    print(f"Sending SMS: {message}")

class NotificationService:
  def __init__(self,message:Messanger):
    self.message=message
  def send_notification(self,message):
    self.message.send_message(message)


email_message=Email()
sms_message=SMS()
notification_service = NotificationService(email_message)
notification_service.send_notification("Hello via Email!")

notification_service = NotificationService(sms_message)
notification_service.send_notification("Hello via SMS!")



Sending email: Hello via Email!
Sending SMS: Hello via SMS!


finall taks: we have this awful code

In [ ]:
class User:
    def __init__(self, name, email, password):
        self.name = name
        self.email = email
        self.password = password

    def login(self):
        # authenticate user
        if self.password == 'secret':
            print("Logged in")
        else:
            print("Login failed")

    def send_email(self, message):
        # send email
        print(f"Sending email to {self.email}: {message}")

    def display_user(self):
        print(f"User: {self.name}, Email: {self.email}")

class PremiumUser(User):
    def login(self):
        # authenticate premium user differently
        if self.password == 'premiumsecret':
            print("Premium user logged in")
        else:
            print("Login failed")

# Example usage
user = User("John Doe", "john@example.com", "secret")
user.login()
user.send_email("Hello John!")
user.display_user()

premium_user = PremiumUser("Jane Doe", "jane@example.com", "premiumsecret")
premium_user.login()
premium_user.send_email("Hello Jane!")
premium_user.display_user()


Logged in
Sending email to john@example.com: Hello John!
User: John Doe, Email: john@example.com
Premium user logged in
Sending email to jane@example.com: Hello Jane!
User: Jane Doe, Email: jane@example.com


let rewrite it considering the SOLID principles

In [ ]:
from abc import ABC , abstractmethod

class LogginService(ABC):
  @abstractmethod
  def loggin(self,password):
    pass
class EmailService(ABC):
  @abstractmethod
  def send_email(self,email,message):
    pass
class DisplayService(ABC):
  @abstractmethod
  def display_user(self,name,email):
    pass

class User(LogginService,EmailService,DisplayService):
  def loggin(self,password):
    if password == 'secret':
      print("Logged in")
    else:
      print("Login failed")
  def send_email(self,email,message):
    print(f"Sending email to {email}: {message}")
  def display_user(self,name,email):
    print(f"User: {name}, Email: {email}")

class PremiumUser(LogginService):
  def loggin(self,password):
    if password == 'premiumsecret':
      print("Premium user logged in")
    else:
      print("Login failed")

user = User()
user.loggin('secret')
user.send_email("john@example.com","Hello John!")
user.display_user("John Doe", "john@example.com")


premium_user = PremiumUser()
premium_user.loggin("premiumsecret")

Logged in
Sending email to john@example.com: Hello John!
User: John Doe, Email: john@example.com
Premium user logged in


the above code is better than the first one but it can get better

In [ ]:
from abc import ABC , abstractmethod

class LogginService(ABC):
  @abstractmethod
  def loggin(self,password):
    pass

class UserLoginService(LogginService):
  def loggin(self,password):
    if password == 'secret':
      print("Logged in")
    else:
      print("Login failed")

class PremiumUserLoginService(LogginService):
  def loggin(self,password):
    if password == 'premiumsecret':
      print("Premium user logged in")
    else:
      print("Login failed")


class EmailService(ABC):
  @abstractmethod
  def send_email(self,email,message):
    pass
class UserEmailService(EmailService):
  def send_email(self,email,message):
    print(f"Sending email to {email}: {message}")



class DisplayService(ABC):
  @abstractmethod
  def display_user(self,name,email):
    pass
class UserDisplayService(DisplayService):
  def display_user(self,name,email):
    print(f"User: {name}, Email: {email}")



class User(UserLoginService,UserEmailService,UserDisplayService):
  def __init__(self,user_login_service:UserLoginService,user_email_service:UserEmailService,user_display_service:UserDisplayService):
    self.user_login_service=user_login_service
    self.user_email_service=user_email_service
    self.user_display_service=user_display_service

  def loggin(self,password):
    self.user_login_service.loggin(password)
  def send_email(self,email,message):
    self.user_email_service.send_email(email,message)
  def display_user(self,name,email):
    self.user_display_service.display_user(name,email)

class PremiumUser(PremiumUserLoginService):
    def __init__(self,user_login_service:PremiumUserLoginService):
    self.user_login_service=user_login_service
  def loggin(self,password):
    self.user_login_service.loggin(password)



now this code still breaks the DIP cause the high level which is User and PremiumUser are dependening on lowe levels

In [1]:
from abc import ABC, abstractmethod

# Abstract base class for logging service
class LoginService(ABC):
    @abstractmethod
    def login(self, password):
        pass

# Concrete implementation of LoginService for regular users
class UserLoginService(LoginService):
    def login(self, password):
        if password == 'secret':
            print("Logged in")
        else:
            print("Login failed")

# Concrete implementation of LoginService for premium users
class PremiumUserLoginService(LoginService):
    def login(self, password):
        if password == 'premiumsecret':
            print("Premium user logged in")
        else:
            print("Login failed")

# Abstract base class for email service
class EmailService(ABC):
    @abstractmethod
    def send_email(self, email, message):
        pass

# Concrete implementation of EmailService
class UserEmailService(EmailService):
    def send_email(self, email, message):
        print(f"Sending email to {email}: {message}")

# Abstract base class for display service
class DisplayService(ABC):
    @abstractmethod
    def display_user(self, name, email):
        pass

# Concrete implementation of DisplayService
class UserDisplayService(DisplayService):
    def display_user(self, name, email):
        print(f"User: {name}, Email: {email}")

# User class using composition
class User:
    def __init__(self, login_service: LoginService, email_service: EmailService, display_service: DisplayService):
        self.login_service = login_service
        self.email_service = email_service
        self.display_service = display_service

    def login(self, password):
        self.login_service.login(password)

    def send_email(self, email, message):
        self.email_service.send_email(email, message)

    def display_user(self, name, email):
        self.display_service.display_user(name, email)

# PremiumUser class using composition
class PremiumUser:
    def __init__(self, login_service: LoginService):
        self.login_service = login_service

    def login(self, password):
        self.login_service.login(password)

# Example usage
user_login_service = UserLoginService()
user_email_service = UserEmailService()
user_display_service = UserDisplayService()

user = User(user_login_service, user_email_service, user_display_service)
user.login('secret')
user.send_email("john@example.com", "Hello John!")
user.display_user("John Doe", "john@example.com")

premium_user_login_service = PremiumUserLoginService()
premium_user = PremiumUser(premium_user_login_service)
premium_user.login("premiumsecret")


Logged in
Sending email to john@example.com: Hello John!
User: John Doe, Email: john@example.com
Premium user logged in
